---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2017</h1>
<h1 style="font-size: 250%;">Problem Set #3 </h1>
<h1>Issued Wednesday, 3/22/2017; Due Monday, 4/3/2017</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Nick"
    last = "Gooch"
    onyen = "nbgooch"
    pid = "720052029"

---
**Problem #1:** On your midterm you were asked to find all 10-peptide chains whose molecular weight equals a given target (1024) using the following list of residues ordered by their weights. Most of the algorithms proposed were time-consuming with many possible answers.

```python
Residues = [
    (186, 'W'), (163, 'Y'), (156, 'R'), (147, 'F'), (137, 'H'), 
    (131, 'M'), (129, 'E'), (128, '[K/Q]'), (115, 'D'), (114, 'N'),
    (113, '[I/L]'), (103, 'C'), (101, 'T'), (99, 'V'), (97, 'P'), 
    (87, 'S'), (71, 'A'), (57, 'G')
]
```

Now consider a slightly different problem: find the *length* of the *shortest peptide chain* whose molecular weight matches a target. This problem can be solved effciently using *dynamic programming*. In order to see how, consider an analogy between finding the shortest peptide combination of a given weight, and the problem of making *exact change* from an arbitrary set of coinage. 

In the space below given a reccurance relation for the dynamic program for finding the shortest combination of peptides, *DPShortestPeptideLength(target, Residues)*.

           {1 + len(mw(x-calcweight('W'))) }
            .
            .
            .
mw(x) = min{1 + len(mw(x-clacweight('L')))}
            .
            .
            .
           {1 + len(mw(x-calcweight('G')))}
        

                      {if x - calweight('W') > 0:   }
                      {  = x - calweight('W')      }
x - calweight('W') =  {else x - calcweight('W') < 0:}
                      {  = infintiy                 }

---
**Problem #2:** It is inefficient to create and fully populate a table for DPShortestPeptideCombination(target, Residues) where every molecular weight less than your given target, as you would for computing exact change. Suggest a more efficient means for representing and populating the memoization of intermediate results needed for *DPShortestPeptideLength(target, Residues)*.

---
**Problem #3: Programming Problem**

On the midterm you were asked to provide a strategy for finding the length of the *longest repeated* *k*-mer (i.e. it appears at least twice) in the [Mitochondrial sequence of the house mouse](http://csbio.unc.edu/mcmillan/Comp555S17/data/MusMusculusMT.fa). When discussing the midterm solutions I provided a dynamic programming approach to solve this problem for linear genomes. However, as you may recall, the mouse mitochondria is a circular genome with 16399 bases.

Now you are asked to write your own dynamic programming solution that works for a circular genome. Enter your code in the space provided below. It should execute on the given mitochondrial sequence.

In [2]:
# import numpy 

def loadFasta(filename):
    fp = open(filename, 'rb')
    data = fp.read().split('>')
    fp.close
    data.pop(0)
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        sequences.append(''.join(lines))
    return (headers, sequences)

def maxRepeatedKmer(seq):
    N_orig = (len(seq)+1)/2
    N = len(seq) 
    score = numpy.zeros((N+1,N+1), dtype=numpy.int) 
    for i in xrange(0,N-1): 
        for j in xrange(i+1,N): 
            if (seq[i] == seq[j]): 
                if (score[i-1,j-1] + 1 <= N_orig - 1):
                    if (j - score[i-1,j-1] < N_orig):
                        score[i,j] = score[i-1,j-1] + 1 
            else: 
                score[i,j] = 0 
    return score 

# mySeq = 'AATAA'
head, mySeq = loadFasta('txt/musMito.fa')
mySeq = mySeq[0]
mySeq = mySeq*2
mySeq = mySeq[:-1]
%time maxRepeatedKmer(mySeq)
score = maxRepeatedKmer(mySeq)
i,j = numpy.unravel_index(score.argmax(), score.shape) 
k = score[i,j] 
print "k =", k, mySeq[i-k+1:i+1], mySeq[j-k:j]

NameError: global name 'numpy' is not defined

**Problem #4:** There may be multiple *longest repeated k*-mers in a given sequence. There can be different substrings of the same length and a given substring might be repeated more than two times. Suppose that rather that returning just the length of the longest repeated *k*-mer, as you were asked in **Problem #3**, you were asked to return a comprehensive list of all distinct *longest repeated k*-mers and how many times it occurs. Explain below how your would modify your solution from **Problem #3** to achieve this objective. 

**Problem #5:** In the space provided below provide an example of a linear sequence where no *5*-mer is repeated and it is as long as possible.

In [1]:
import itertools

class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in xrange(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.iterkeys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print "There is no path!"
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in xrange(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in xrange(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in xrange(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in xrange(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.iteritems():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result

def kmerFreq(k, sequence):
    """ returns the count of all k-mers in sequence as a dictionary"""
    kmerCount = {}
    for i in xrange(len(sequence)-k+1):
        kmer = sequence[i:i+k]
        kmerCount[kmer] = kmerCount.get(kmer,0)+1
    return kmerCount


kmers = [''.join(t) for t in itertools.product('ACGT', repeat=5)]

nodes = sorted(set([code[:-1] for code in kmers] + [code[1:] for code in kmers]))
G2 = Graph(nodes)
for code in kmers:
   # Here I give each edge a label
   G2.addEdge(code[:-1],code[1:],code)
path = G2.eulerianPath()

ss = G2.eulerEdges(path)[0]
for edge in G2.eulerEdges(path[1:]):
    ss += edge[-1:]
totalK = kmerFreq(5, ss)
print 'Superstring: {}'.format(ss) 
print 'Total # kmers: {}'.format(len(totalK))
print 'Length of Superstring: {}'.format(len(ss))

Superstring: AAAAACCCCCGGGGGTTTTTGTTTGGTTGGGTGTTGTGTGGTGGGGCGGGTCGGGCCGGTGCGGTTCGGTCCGGCTCGGCCCGTGGCGTGTCGTGCCGTTGCGTTTCGTTCCGTCTCGTCCCGCGGCGCGTCGCGCCGCTGCGCTTCGCTCCGCCCCTGGGCTGGTCTGGCCTGTGCTGTTCTGTCCTGCCCTTGGCTTGTCTTGCCTTTGCTTTTCTTTCCTTCCCTCGCCTCTGCTCTTCTCTCCTCCCCACCCGACCCTACCCAACCGCACCGGACCGTACCGAACCTCACCTGACCTTACCTAACCAGACCATACCAAACGCCACGCGACGCTACGCAACGGCACGGGACGGTACGGAACGTCACGTGACGTTACGTAACGAGACGATACGAAACTCCACTCGACTCTACTCAACTGCACTGGACTGTACTGAACTTCACTTGACTTTACTTAACTAGACTATACTAAACACCACACGACACTACACAACAGCATAGCACAGGATAGGACAGTATAGTACAGAACATCACATGACATTACATAACAAAAGCCCAGCCGAGCCTAGCCAAGCGCAGCGGAGCGTAGCGAAGCTCAGCTGAGCTTAGCTAAGCAAAGGCCAGGCGAGGCTAGGCAAGGGCAGGGGAGGGTAGGGAAGGTCAGGTGAGGTTAGGTAAGGAAAGTCCAGTCGAGTCTAGTCAAGTGCAGTGGAGTGTAGTGAAGTTCAGTTGAGTTTAGTTAAGTAAAGACAAGAGCAGAGGAGAGTAGAGAAGATCAGATGAGATTAGATAAGAAAATCCCATCCGATCCTATCCAATCGCATCGGATCGTATCGAATCTCATCTGATCTTATCTAATCAAATGCCATGCGATGCTATGCAATGGCATGGGATGGTATGGAATGTCATGTGATGTTATGTAATGAAATTCCATTCGATTCTATTCAATTGCATTGGATTGTATTGAATTTCATTTGATTTTATTTA

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set